# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


## Data load

In [1]:
! hadoop fs -ls -h /data

Found 2 items
-rw-r--r--   1 root supergroup     30.7 M 2023-09-24 20:38 /data/clickstream.csv
drwxr-xr-x   - root supergroup          0 2023-09-24 20:38 /data/transactions


In [2]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-10 09:55:11,593 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
events = se.read.option("delimiter", "\t").csv("hdfs:///data/clickstream.csv", header=True)  # в первой строке у нас заголовок
events.limit(10).toPandas()

,user_id,session_id,event_type,event_page,timestamp
0,562,507,page,main,1695584127
1,562,507,event,main,1695584134
2,562,507,event,main,1695584144
3,562,507,event,main,1695584147
4,562,507,wNaxLlerrorU,main,1695584154
5,562,507,event,main,1695584154
6,562,507,event,main,1695584154
7,562,507,event,main,1695584160
8,562,507,page,rabota,1695584166
9,562,507,event,rabota,1695584174


As we need to extract paths for each user session, in every solution we will use user_id and session_id, separtated by _, as the unique id of a user session.

## Spark SSL solution

In [4]:
events.registerTempTable('events')
event_views = se.table("events")
event_views.select('*').show()

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
|    562|       507|       event|    rabota|1695584181|
|    562|       507|       event|    rabota|1695584189|
|    562|       507|        page|      main|1695584194|
|    562|       507|       event|      main|1695584204|
|    562|       507|       event|      main|1695

Here we will join two queries to events table: one for extracting the timestamps of errors and one for extracting routes. To collect all valid pages to a path, we will use collect_list function over a window.

In [11]:
paths_sql = se.sql("""
select concat_ws('-', route_parts) as route, count(id) as cnt
from (
    select distinct
        CONCAT(user_id, '_',  session_id) as id, collect_list(event_page) over (partition by CONCAT(user_id, '_',  session_id) order by timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as route_parts
    from events
    left join (
        select CONCAT(user_id, '_',  session_id) as id, min(int(timestamp)) as min_timestamp
        from events
        where event_type not in ('page', 'event')
        group by CONCAT(user_id, '_',  session_id)
    ) errors on CONCAT(events.user_id, '_',  events.session_id) == errors.id
    where events.event_type = 'page' and (errors.min_timestamp is null or int(events.timestamp) < errors.min_timestamp)
) paths
group by route
order by cnt desc
""").limit(30)
paths_sql.show(30)

+--------------------+----+
|               route| cnt|
+--------------------+----+
|                main|8090|
|        main-archive|1096|
|         main-rabota|1039|
|       main-internet| 880|
|          main-bonus| 865|
|           main-news| 760|
|        main-tariffs| 669|
|         main-online| 584|
|          main-vklad| 514|
| main-archive-rabota| 167|
| main-rabota-archive| 167|
|  main-bonus-archive| 139|
|   main-rabota-bonus| 137|
|    main-news-rabota| 134|
|   main-bonus-rabota| 133|
|main-archive-inte...| 131|
|    main-rabota-news| 129|
|main-internet-rabota| 128|
|   main-archive-news| 125|
|main-internet-arc...| 123|
|main-rabota-internet| 123|
|  main-archive-bonus| 117|
|main-tariffs-inte...| 114|
| main-internet-bonus| 114|
|   main-news-archive| 112|
|  main-news-internet| 108|
|main-archive-tariffs| 103|
|  main-internet-news| 102|
|main-tariffs-archive| 102|
|           main-main|  94|
+--------------------+----+



In [24]:
with open("paths_sql.txt", "w") as file_sql:
    for item in paths_sql.collect():
        file_sql.write(item['route'] + '\t' + str(item['cnt']) + '\n')

## Spark RDD solution

Here we will create two rdds - one to get a dictionary of user session ids to error timestamp of a given session, and one - to get the routes. The function sort_by_time() is used to map each user session to paths sorted in the order of timestamps.

In [25]:
rdd = events.rdd
errors_rdd = (
    rdd
    .filter(lambda x: x['event_type'] not in ('page', 'event'))
    .map(lambda x: (x['user_id'] + '_' + x['session_id'], int(x['timestamp'])))
    .reduceByKey(lambda a, b: min(a, b))
)
errors = dict(errors_rdd.collect())

In [26]:
def sort_by_time(it):
    it_sorted = sorted(list(it), key=lambda x: x[0])
    return '-'.join(map(lambda x: x[1], it_sorted))

paths = (
    rdd
    .filter(lambda x: (x['event_type'] == 'page'))
    .map(lambda x: (x['user_id'] + '_' + x['session_id'], (int(x['timestamp']), x['event_page'])))
    .filter(lambda x: (x[0] not in errors) or (x[1][0] < errors[x[0]]))
    .groupByKey()
    .mapValues(sort_by_time)
    .map(lambda x: (x[1], x[0]))
    .countByKey()
)
paths_rdd = sorted(paths.items(), key=lambda x: -x[1])[:30]
paths_rdd

[('main', 8090),
 ('main-archive', 1096),
 ('main-rabota', 1039),
 ('main-internet', 880),
 ('main-bonus', 865),
 ('main-news', 760),
 ('main-tariffs', 669),
 ('main-online', 584),
 ('main-vklad', 514),
 ('main-rabota-archive', 167),
 ('main-archive-rabota', 167),
 ('main-bonus-archive', 139),
 ('main-rabota-bonus', 137),
 ('main-news-rabota', 134),
 ('main-bonus-rabota', 133),
 ('main-archive-internet', 131),
 ('main-rabota-news', 129),
 ('main-internet-rabota', 128),
 ('main-archive-news', 125),
 ('main-rabota-internet', 123),
 ('main-internet-archive', 123),
 ('main-archive-bonus', 117),
 ('main-tariffs-internet', 114),
 ('main-internet-bonus', 114),
 ('main-news-archive', 112),
 ('main-news-internet', 108),
 ('main-archive-tariffs', 103),
 ('main-tariffs-archive', 102),
 ('main-internet-news', 102),
 ('main-main', 94)]

In [27]:
with open("paths_rdd.txt", "w") as file_rdd:
    for item in paths_rdd:
        file_rdd.write(item[0] + '\t' + str(item[1]) + '\n')

## Spark DF solution

Here we will use three dataframes - errors_df for error timestampts, events_df for extracting page events mapped to unique user session ids, and path dataframe, which joins the first two. We will also need a window object to sort the pages correctly within each user session. 

In [29]:
from pyspark.sql import functions as F

errors_df = (
    events
    .where(~(F.col('event_type').isin('page', 'event')))
    .select(F.concat(F.col('user_id'), F.lit('_'), F.col('session_id')).alias('id'), F.col("timestamp").cast("int").alias("timestamp"))
    .groupby('id')
    .agg(F.min("timestamp").alias("min_timestamp"))
)
errors = errors_df.toPandas().set_index('id').to_dict(orient = 'list')
errors_df.limit(10).toPandas()

,id,min_timestamp
0,996_797,1695587393
1,3646_465,1695597176
2,2547_608,1695645778
3,2649_476,1695861384
4,112_4,1695876444
5,4512_949,1695949026
6,4667_87,1695994992
7,4229_942,1696006992
8,4623_179,1696070855
9,2952_9,1696181787


In [31]:
from pyspark.sql.window import Window

events_df = (
    events
    .where(F.col('event_type') == 'page')
    .select(F.concat(F.col('user_id'), F.lit('_'), F.col('session_id')).alias('id'), F.col("timestamp").cast("int").alias("timestamp"), F.col('event_page'))
)
window = Window.partitionBy('id').orderBy("timestamp").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
paths = (
    events_df
    .join(errors_df, events_df.id == errors_df.id, 'leftouter')
    .where(F.col('min_timestamp').isNull() | (F.col('timestamp') < F.col('min_timestamp')))
    .select(events_df.id, events_df.event_page, events_df.timestamp)
    .withColumn('route_parts', F.collect_list(F.col('event_page')).over(window))
    .withColumn('route', F.concat_ws('-', F.col('route_parts')))
    .select('route', 'id')
    .distinct()
    .groupby('route')
    .agg(F.count('id').alias("cnt"))
    .sort(F.desc("cnt"))
)
paths_df = paths.limit(30).toPandas()
paths_df

,route,cnt
0,main,8090
1,main-archive,1096
2,main-rabota,1039
3,main-internet,880
4,main-bonus,865
5,main-news,760
6,main-tariffs,669
7,main-online,584
8,main-vklad,514
9,main-rabota-archive,167


In [37]:
with open("paths_df.txt", "w") as file_df:
    for index, item in paths_df.iterrows():
        file_df.write(item['route'] + '\t' + str(item['cnt']) + '\n')